The goal of this file is to vectorize sentences.

In [28]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec

In [29]:
# Load the data
train = pd.read_excel('datasets/all_train_data.xlsx')

In [30]:
# Remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
train['avis_en_cleaned'] = train['avis_en'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

In [31]:
# Vectorize the data
# Tokenize the data
train['tokenized'] = train['avis_en'].apply(lambda x: str(x).split())

# Create the model
model = word2vec.Word2Vec(train['tokenized'], min_count=1, vector_size=400)

In [32]:
def compute_embedding(text, model):
    return np.mean([model.wv[word] for word in str(text).split()], axis=0)

# Compute the embedding
train['embedding'] = train['avis_en'].apply(lambda x: compute_embedding(x, model))

In [34]:
train

,note,auteur,avis,assureur,produit,type,date_publication,date_exp,avis_en,avis_cor,avis_cor_en,avis_en_cleaned,tokenized,embedding
0,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,train,06/09/2021,01/09/2021,"Best insurance, price, solutions, listening, s...",NaN,NaN,"Best insurance, price, solutions, listening, s...","[Best, insurance,, price,, solutions,, listeni...","[-0.028961312, -0.050040387, 0.15890452, 0.208..."
1,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,train,03/05/2021,01/05/2021,"I am generally satisfied, except that you have...",NaN,NaN,"I generally satisfied, except problem website,...","[I, am, generally, satisfied,, except, that, y...","[0.18049645, -0.21754728, 0.20137589, 0.026057..."
2,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,train,21/03/2021,01/03/2021,Very affordable price Several options are avai...,NaN,NaN,Very affordable price Several options availabl...,"[Very, affordable, price, Several, options, ar...","[-0.08154435, -0.08826422, 0.3592564, 0.135076..."
3,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,train,10/06/2021,01/06/2021,"I satisfy the service, a very fast response fr...",NaN,NaN,"I satisfy service, fast response service. I th...","[I, satisfy, the, service,, a, very, fast, res...","[0.024485236, -0.21532096, 0.027519096, 0.1744..."
4,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,train,29/01/2017,01/01/2017,"Customer for more than 25 years, very disappoi...",NaN,NaN,"Customer 25 years, disappointed ""mutual"" longe...","[Customer, for, more, than, 25, years,, very, ...","[-0.025798883, -0.29428703, 0.055660814, 0.276..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,4,krys34-96609,J'ai jamais eu de soucis et toujours etait bie...,Assur O'Poil,animaux,train,24/08/2020,01/08/2020,I never had problems and always was well reimb...,NaN,NaN,I never problems always well reimbursed dog ​​...,"[I, never, had, problems, and, always, was, we...","[0.12127331, -0.25959507, 0.15001829, 0.139646..."
23996,3,fati-98297,Après 37 ans je quitte la Macif à la suite de ...,MACIF,auto,train,03/10/2020,01/10/2020,After 37 years I leave the Macif following sev...,NaN,NaN,After 37 years I leave Macif following several...,"[After, 37, years, I, leave, the, Macif, follo...","[-0.016296739, -0.23654395, 0.11040547, 0.0977..."
23997,1,jeremy59-96188,Je suis propriétaire d'une flotte de véhicules...,AXA,flotte-automobile,train,11/08/2020,01/08/2020,I own a fleet of motor vehicles which was insu...,NaN,NaN,I fleet motor vehicles insured Axa Insurance m...,"[I, own, a, fleet, of, motor, vehicles, which,...","[-0.10706867, -0.2895521, 0.058579843, 0.07511..."
23998,1,mic-54375,Mutuelle à fuir. Chère. Remboursements insuffi...,Mgen,sante,train,29/04/2017,01/04/2017,Mutual to flee. Dear. Insufficient reimburseme...,NaN,NaN,Mutual flee. Dear. Insufficient reimbursements...,"[Mutual, to, flee., Dear., Insufficient, reimb...","[-0.0067299423, -0.18810418, 0.25117242, 0.244..."
